In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')

In [4]:
len(df.investment_id.unique())

3579

In [5]:
max(df.investment_id.unique())

3773

here is a little different. for the in the origional tensorflow. the data is made from the len+1 yet in the previous test it was made from max+1
I do not think just use the len is a good idea for some of the data will only appear on the test set so i think it is a little unsafe.
this experiment is based on the len instead of max

In [6]:
feature = df.drop(['row_id', 'time_id', 'target',"investment_id"], axis=1)
id=df["investment_id"]
y=df["target"]

In [7]:
import torch
from torch.utils.data import DataLoader,Dataset

the orginal code actually uses mutiple models to do the predictions and decides to use PCA to get them all together
the 3 models must share the same structure and the difference might lies on the loss function, optimiser and the train dataset.
I am going to create 3 models: loss has 3 variants and we can do it in different batch size. 

FYI, poor ensemble learning

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature=torch.from_numpy(feature.values).to(device)
id=torch.from_numpy(id.values).to(device)
y=torch.from_numpy(y.values).to(device)
class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to("cuda")
        self.y = y.squeeze().to("cuda")
        self.id = id.squeeze().to(torch.int64).to("cuda")
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_train=FinData(feature=feature,id=id,y=y)
DataLoader1=DataLoader(Dataset_train,batch_size=256,shuffle=True)
DataLoader2=DataLoader(Dataset_train,batch_size=1024,shuffle=True)


In [9]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,32)
        self.linear1=nn.Linear(32,64)
        self.linear2=nn.Linear(64,64)
        self.linear3=nn.Linear(300,256)
        self.linear4=nn.Linear(256,256)
        self.linear5=nn.Linear(320,512)
        self.linear6=nn.Linear(512,128)
        self.linear7=nn.Linear(128,32)
        self.linear8=nn.Linear(32,1)
      

    def forward(self, feature, id):
        id=self.embedding(id).squeeze()
        id=self.linear1(id).squeeze()
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)


        feature=self.linear3(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)

        x=torch.cat((id,feature),1).to("cuda")
        x=self.linear5(x).squeeze()
        x=x*torch.sigmoid(x)
        x=self.linear6(x).squeeze()
        x=x*torch.sigmoid(x)

        x=self.linear7(x).squeeze()
        x=x*torch.sigmoid(x)

        x=self.linear8(x).squeeze()
        x=40*torch.sigmoid(x)-20




       # linear_result=self.act(self.linear2(linear_result))
        #result=self.linear3(result)
        #result=40*torch.sigmoid(result)-20
        return x.squeeze()

model1_1 = NeuralNetwork().to("cuda")
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to("cuda")
model1_2=model1_2.double()
model1_3 = NeuralNetwork().to("cuda")
model1_3=model1_3.double()
model1_4 = NeuralNetwork().to("cuda")
model1_4=model1_4.double()

model2_1 = NeuralNetwork().to("cuda")
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to("cuda")
model2_2=model2_2.double()
model2_3 = NeuralNetwork().to("cuda")
model2_3=model2_3.double()
model2_4 = NeuralNetwork().to("cuda")
model2_4=model2_4.double()



In [10]:
learning_rate = 1e-3
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer1_3 = torch.optim.Adam(model1_3.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer1_4 = torch.optim.Adam(model1_4.parameters(), lr=learning_rate,weight_decay=1e-5)


optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_2 = torch.optim.Adam(model2_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_3 = torch.optim.Adam(model2_3.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_4 = torch.optim.Adam(model2_4.parameters(), lr=learning_rate,weight_decay=1e-5)

loss_fn1=torch.nn.MSELoss()

In [11]:
from tqdm import tqdm

In [12]:
def MAPELoss(output, target):
  return torch.mean(torch.abs((target - output) / target))    

def RPDLoss(output, target):
  return torch.mean(torch.abs(target - output) / ((torch.abs(target) + torch.abs(output)) / 2))    

In [13]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = loss_fn1(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_2(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = -torch.corrcoef(torch.stack((pred,y)))[0,1]
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_3(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = MAPELoss(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_4(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        # Compute prediction and loss
        pred = model(feature,id)
        loss = RPDLoss(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [14]:
def test(dataloader,model):
    averagescore=0
    i=0
    for id, feature,y in dataloader:
        i=i+1
        pred=model(feature,id)
        score=torch.corrcoef(torch.stack((pred,y)))[0,1]
        averagescore+=score
    averagescore/=i
    return averagescore
        

In [15]:
for i in range(5):
    print("epoch{}/{}".format(i+1,5))
    train_1(DataLoader1,model1_1,optimizer1_1)
    train_2(DataLoader1,model1_2,optimizer1_2)
    train_3(DataLoader1,model1_3,optimizer1_3)
    train_4(DataLoader1,model1_4,optimizer1_4)

    train_1(DataLoader2,model2_1,optimizer2_1)
    train_2(DataLoader2,model2_2,optimizer2_2)
    train_3(DataLoader2,model2_3,optimizer2_3)
    train_4(DataLoader2,model2_4,optimizer2_4)

epoch1/5


100%|██████████| 3068/3068 [00:55<00:00, 55.30it/s]


epoch2/5


100%|██████████| 3068/3068 [00:54<00:00, 55.86it/s]


epoch3/5


100%|██████████| 3068/3068 [00:53<00:00, 57.05it/s]


epoch4/5


100%|██████████| 3068/3068 [00:48<00:00, 63.15it/s]


epoch5/5


100%|██████████| 3068/3068 [00:48<00:00, 63.23it/s]


In [16]:
torch.save(model1_1, 'tensorflow model/model1_1.pth')
torch.save(model1_2, 'tensorflow model/model1_2.pth')
torch.save(model1_3, 'tensorflow model/model1_3.pth')
torch.save(model1_4, 'tensorflow model/model1_4.pth')

torch.save(model2_1, 'tensorflow model/model2_1.pth')
torch.save(model2_2, 'tensorflow model/model2_2.pth')
torch.save(model2_3, 'tensorflow model/model2_3.pth')
torch.save(model2_4, 'tensorflow model/model2_4.pth')



In [17]:
model1_1 = NeuralNetwork().to("cuda")
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to("cuda")
model1_2=model1_2.double()

model2_1 = NeuralNetwork().to("cuda")
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to("cuda")
model2_2=model2_2.double()


model3_1 = NeuralNetwork().to("cuda")
model3_1=model3_1.double()
model3_2 = NeuralNetwork().to("cuda")
model3_2=model3_2.double()


model1_1=torch.load('tensorflow model/model1_1.pth')
model1_2=torch.load('tensorflow model/model1_2.pth')
model2_1=torch.load('tensorflow model/model2_1.pth')
model2_2=torch.load('tensorflow model/model2_2.pth')
model3_1=torch.load('tensorflow model/model3_1.pth')
model3_2=torch.load('tensorflow model/model3_2.pth')

In [18]:
test(DataLoader1,model1_1)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 16.00 GiB total capacity; 14.35 GiB already allocated; 0 bytes free; 14.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
len(m)

7

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)

In [ ]:
 pca.fit_transform(m)


array([[-0.60609153],
       [-0.60609153],
       [ 0.80812204],
       [-0.60609153],
       [-0.60609153],
       [ 2.2223356 ],
       [-0.60609153]])